<a href="https://colab.research.google.com/github/Melckykaisha/Movie_recommender_system/blob/main/HMM_POS_Tagger.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hidden Markov Model (HMM) for Part-of-Speech Tagging
This notebook demonstrates how to build a POS tagger from scratch using Hidden Markov Models (HMMs). We train on the Brown corpus (universal tagset) via NLTK, estimate probabilities, implement the Viterbi algorithm, and evaluate accuracy.

In [1]:
import nltk
from collections import defaultdict, Counter
import math, random

# Ensure required corpora are downloaded (Brown + universal tagset)
nltk.download('brown')
nltk.download('universal_tagset')

tagged_sents = nltk.corpus.brown.tagged_sents(tagset='universal')
print("Total tagged sentences in Brown (universal):", len(tagged_sents))

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Unzipping taggers/universal_tagset.zip.


Total tagged sentences in Brown (universal): 57340


In [ ]:

random.seed(42)

# Normalize words to lowercase
def normalize_tagged_sents(tagged_sents):
    return [[(w.lower(), t) for (w,t) in sent] for sent in tagged_sents]

tagged_sents = normalize_tagged_sents(tagged_sents)

# Split into train/test
split_ratio = 0.9
split_idx = int(len(tagged_sents) * split_ratio)
train_sents = tagged_sents[:split_idx]
test_sents = tagged_sents[split_idx:]
print(f"Train sentences: {len(train_sents)}, Test sentences: {len(test_sents)}")


Train sentences: 51606, Test sentences: 5734


In [ ]:

def train_hmm(tagged_sents, k=1.0):
    tags = set()
    vocab = set()
    initial_counts = Counter()
    transition_counts = defaultdict(Counter)
    emission_counts = defaultdict(Counter)
    total_sentences = 0

    for sent in tagged_sents:
        if not sent:
            continue
        total_sentences += 1
        words, tg = zip(*sent)
        tags.update(tg)
        vocab.update(words)
        initial_counts[tg[0]] += 1
        for i in range(len(sent)):
            emission_counts[tg[i]][words[i]] += 1
            if i > 0:
                transition_counts[tg[i-1]][tg[i]] += 1

    tags = sorted(tags)
    V = len(vocab)
    T = len(tags)

    for tag in tags:
        emission_counts[tag]['<UNK>'] += 0

    init_probs, trans_probs, emit_probs = {}, defaultdict(dict), defaultdict(dict)
    total_init = sum(initial_counts.values())

    for tag in tags:
        init_probs[tag] = (initial_counts[tag] + k) / (total_init + k*T)

    for prev in tags:
        tot = sum(transition_counts[prev].values())
        for curr in tags:
            trans_probs[prev][curr] = (transition_counts[prev][curr] + k) / (tot + k*T)

    for tag in tags:
        tot_e = sum(emission_counts[tag].values())
        for word, cnt in emission_counts[tag].items():
            emit_probs[tag][word] = (cnt + k) / (tot_e + k*(V+1))
        emit_probs[tag]['<UNK>'] = (emission_counts[tag].get('<UNK>', 0) + k) / (tot_e + k*(V+1))

    log_init = {tag: math.log(init_probs[tag]) for tag in tags}
    log_trans = {prev: {curr: math.log(trans_probs[prev][curr]) for curr in tags} for prev in tags}
    log_emit = {tag: {w: math.log(p) for w,p in emit_probs[tag].items()} for tag in tags}

    return {
        'tags': tags, 'vocab': vocab,
        'init_probs': init_probs, 'trans_probs': trans_probs, 'emit_probs': emit_probs,
        'log_init': log_init, 'log_trans': log_trans, 'log_emit': log_emit
    }

model = train_hmm(train_sents, k=1.0)
print("Number of tags:", len(model['tags']), "Vocabulary size:", len(model['vocab']))


Number of tags: 12 Vocabulary size: 47711


In [ ]:

NEG_INF = -1e12

def viterbi(sentence_words, model):
    T = len(sentence_words)
    tags = model['tags']
    log_init, log_trans, log_emit, vocab = model['log_init'], model['log_trans'], model['log_emit'], model['vocab']

    words = [w.lower() for w in sentence_words]
    words_mapped = [w if w in vocab else '<UNK>' for w in words]

    V = [defaultdict(lambda: NEG_INF) for _ in range(T)]
    backpointer = [defaultdict(lambda: None) for _ in range(T)]

    # Initialization
    for tag in tags:
        e = log_emit[tag].get(words_mapped[0], log_emit[tag]['<UNK>'])
        V[0][tag] = log_init[tag] + e
        backpointer[0][tag] = None

    # Recursion
    for t in range(1, T):
        for curr in tags:
            e = log_emit[curr].get(words_mapped[t], log_emit[curr]['<UNK>'])
            best_prob, best_prev = NEG_INF, None
            for prev in tags:
                prob = V[t-1][prev] + log_trans[prev][curr] + e
                if prob > best_prob:
                    best_prob, best_prev = prob, prev
            V[t][curr] = best_prob
            backpointer[t][curr] = best_prev

    best_final = max(tags, key=lambda tag: V[T-1][tag])
    best_path = [best_final]
    for t in range(T-1, 0, -1):
        best_path.append(backpointer[t][best_path[-1]])
    best_path.reverse()
    return best_path


In [ ]:

def evaluate_hmm(test_sents, model, max_sentences=None):
    total, correct, nsent = 0, 0, 0
    for sent in test_sents:
        words, gold_tags = zip(*sent)
        pred_tags = viterbi(words, model)
        total += len(gold_tags)
        correct += sum(1 for gt, pt in zip(gold_tags, pred_tags) if gt == pt)
        nsent += 1
        if max_sentences and nsent >= max_sentences:
            break
    return correct/total if total else 0.0

hmm_acc = evaluate_hmm(test_sents, model)
print(f"HMM tagger accuracy on test set: {hmm_acc:.4f}")


HMM tagger accuracy on test set: 0.9319


In [ ]:

example_sent = test_sents[0]
words, gold_tags = zip(*example_sent)
pred_tags = viterbi(words, model)

print("Sentence:", words)
print("Gold tags:", gold_tags)
print("Predicted tags:", pred_tags)


Sentence: ('he', 'was', 'about', '50', 'years', 'old', '.')
Gold tags: ('PRON', 'VERB', 'ADV', 'NUM', 'NOUN', 'ADJ', '.')
Predicted tags: ['PRON', 'VERB', 'ADP', 'NUM', 'NOUN', 'ADJ', '.']


# Task
Visualize the performance of the Hidden Markov Model (HMM) on a single sentence from the test set by comparing the gold standard tags to the tags predicted by the HMM.

## Select a sentence

### Subtask:
Choose a sentence from the test set.


**Reasoning**:
Select the first sentence from the test set as the example sentence.



In [ ]:
example_sent = test_sents[0]

## Get gold and predicted tags

### Subtask:
Obtain the gold standard tags and the tags predicted by the HMM for the selected sentence.


**Reasoning**:
Extract the words and gold tags from the example sentence and predict the tags using the Viterbi algorithm.



In [ ]:
words, gold_tags = zip(*example_sent)
pred_tags = viterbi(words, model)

## Prepare data for visualization

### Subtask:
Format the words, gold tags, and predicted tags into a suitable structure for plotting (e.g., a list of tuples or a pandas DataFrame).


**Reasoning**:
Create a pandas DataFrame from the words, gold tags, and predicted tags for easier visualization.



In [ ]:
import pandas as pd

comparison_df = pd.DataFrame({
    'Word': words,
    'Gold Tag': gold_tags,
    'Predicted Tag': pred_tags
})
display(comparison_df)

,Word,Gold Tag,Predicted Tag
0,he,PRON,PRON
1,was,VERB,VERB
2,about,ADV,ADP
3,50,NUM,NUM
4,years,NOUN,NOUN
5,old,ADJ,ADJ
6,.,.,.


## Visualize the results

### Subtask:
Visualize the results by creating a table comparing the gold and predicted tags for each word in the sentence using the `comparison_df` DataFrame.


**Reasoning**:
Print the comparison_df DataFrame to display the table as requested by the subtask.



In [ ]:
display(comparison_df)

,Word,Gold Tag,Predicted Tag
0,he,PRON,PRON
1,was,VERB,VERB
2,about,ADV,ADP
3,50,NUM,NUM
4,years,NOUN,NOUN
5,old,ADJ,ADJ
6,.,.,.


## Summary:

### Data Analysis Key Findings

*   The analysis focused on the first sentence from the test set: "he was about 50 years old ."
*   The Hidden Markov Model (HMM) predicted the tags for this sentence, and these were compared to the gold standard tags.
*   A table was generated to visualize the comparison between gold and predicted tags for each word.
*   The HMM correctly predicted the tags for most words in the sentence.
*   There was one incorrect prediction: the word "about" was predicted as 'ADP' while its gold standard tag was 'ADV'.

### Insights or Next Steps

*   Analyze other sentences with incorrect predictions to identify patterns in the model's errors.
*   Consider feature engineering or model adjustments to improve performance on words like "about" that have ambiguous tag possibilities.
